In [2]:
from underthesea import pos_tag, chunk, word_tokenize
from nltk import RegexpParser, Tree
import pandas as pd
import numpy as np

In [3]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP("./VnCoreNLP-master/VnCoreNLP-1.1.1.jar", annotators="wseg,pos", max_heap_size='-Xmx500m') 

In [4]:
fix = pd.read_csv('./data/noise.csv')['Unnamed: 0'].values # index của các bài báo chứa data bị lỗi

In [5]:
def preprocess(s): #loại bỏ punctuation
    return s.replace(':','').replace(',','').replace('“','').replace('”','').replace('(','').replace(')','').replace('-','').replace('.','')

In [6]:
df = pd.read_csv('./data/filtered_news.csv').drop('Unnamed: 0', axis=1)
text = df.NewsFullContent.apply(lambda s: preprocess(str(s))).replace('', np.nan)
text = text[text.notna()] # loại bỏ các dòng NewsFullContent rỗng
text.head()

0     tên người thực hiện giao dịch phạm nhật vượng...
1    tổ chức thực hiện giao dịch công ty cổ phần cơ...
2    số lượng tại thời điểm  công văn cvssiho xác n...
3    đơn vị tỷ vnđ khoản mục quý iii tháng giá trị ...
4    đơn vị vnd stt chỉ tiêu quý  lũy kế tháng đầu ...
Name: NewsFullContent, dtype: object

In [7]:
text = text.reset_index().drop('index', axis=1).drop(index=fix, axis=0).reset_index().drop('index', axis=1) #loại bỏ các bài báo chứa data bị lỗi
print(text)

                                         NewsFullContent
0       tên người thực hiện giao dịch phạm nhật vượng...
1      tổ chức thực hiện giao dịch công ty cổ phần cơ...
2      số lượng tại thời điểm  công văn cvssiho xác n...
3      đơn vị tỷ vnđ khoản mục quý iii tháng giá trị ...
4      đơn vị vnd stt chỉ tiêu quý  lũy kế tháng đầu ...
...                                                  ...
15687  cổ phiếu ctg  có dấu hiệu mua chủ động với khố...
15688  cổ phiếu ssi  có dấu hiệu bán chủ động với khố...
15689  cổ phiếu pnj  có dấu hiệu bán chủ động với khố...
15690  chốt phiên giao dịch ngày  cổ phiếu vcb dừng ở...
15691  hội đồng quản trị công ty cổ phần tập đoàn mas...

[15692 rows x 1 columns]


In [8]:
def vnpos(text): # part-of-speech tagging
    annotated_text = annotator.annotate(text)
    pos = []
    for dic in annotated_text['sentences'][0]:
        if dic['posTag'] not in ['L','M','E','C','Cc','I','T','Y','Z','X','CH']:
            pos.append(tuple([dic['form'], dic['posTag']]))
        
    return pos

In [30]:
VP = "VP: {<N.>* <V>+ <R>*}"
NP = "NP: {(<N.>+ <A> <E>)? <N.>+ <A>}" #heuristic rule
# VA = "VA : {<V>+ <N.>+ <V>* <A>*}"
# VR = "VR : {<V>+ <N.>+ <V>* <R>*}"
# NP = "NP : {<V>+ <N.>+ <A>*}"
# VP = "VP : {<V>+ <N.>+ <R>*}"
chunker = RegexpParser(VP, NP)

In [31]:
def get_continuous_chunks(text, chunk_func=chunker.parse):
#     text_tokens = word_tokenize(text)
#     tokens_without_sw = [word for word in text_tokens if not word in stopwords]
#     filter_sent = (" ").join(tokens_without_sw)
    pos = pos_tag(text)
    pos_filter = [(a,b) for (a,b) in pos if b not in ['L','M','E','C','Cc','I','T','Y','Z','X','CH']]
    chunked = chunk_func(pos_filter)
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        if type(subtree) == Tree:
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    return continuous_chunk

In [ ]:
candidate_keyword = text.NewsFullContent.apply(lambda s: get_continuous_chunks(s)) 